In [1]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from tqdm import tqdm # Progress bar

from fuzzywuzzy import process
from nltk.tokenize import word_tokenize

//anaconda3/envs/ada/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
output_characters_genders_context=pd.read_csv('DATA/output_characters_genders_context.csv')

Modif initial fct to keep the id column for later merges 

In [3]:
def extract_words(df, id_col, char_name_col, to_extract):
    tokens = pd.Series()
    tagged_tokens = []
    chunks_array = []
    verbs_list = []
    adjs_list = []
    nouns_list = []
    stop_words = set(stopwords.words('english'))

    # Create tqdm progress bar for the loop
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Movies"):
        verbs = []
        adjs = []
        nouns = []
        text = row[to_extract]
        associated_w_text = row[char_name_col]
        movie_id = row[id_col]
        
        if type(text) == str:  # To only keep movies with a summary (ignoring NaN)
            token = [word for word in nltk.word_tokenize(text) if word.lower() not in stop_words]  # Removing stopwords
            tokens[associated_w_text] = token
            tagged_tokens.append((movie_id, associated_w_text, nltk.pos_tag(token)))

    # Tqdm progress bar for the second loop
    for movie_id, associated_w_text, tagged_token in tqdm(tagged_tokens, desc="Processing Tokens", leave=False):
        chunks_array.append((movie_id, associated_w_text, nltk.ne_chunk(tagged_token)))

        verbs = []
        adjs = []
        nouns = []

        # Categorize
        for word, pos_tag in tagged_token:
            if pos_tag in verb_tags:
                verbs.append(word)
            elif pos_tag in adj_tags:
                adjs.append(word)
            elif pos_tag in noun_tags:
                nouns.append(word)

        verbs_list.append((movie_id, associated_w_text, verbs))
        adjs_list.append((movie_id, associated_w_text, adjs))
        nouns_list.append((movie_id, associated_w_text, nouns))

    # Returns lists of all verbs, adjectives, and nouns for each movie and raw chunks for each movie
    return verbs_list, adjs_list, nouns_list, chunks_array

In [ ]:
verbs, adjs, nouns, chunks = extract_words(output_characters_genders_context, "IMDB_ID", "character_name","associated_words")

In [ ]:
# Create DataFrames from the lists
verbs_df = pd.DataFrame(verbs, columns=['IMDB_ID', 'character_name', 'Verbs'])
adjs_df = pd.DataFrame(adjs, columns=['IMDB_ID', 'character_name', 'Adjectives'])
nouns_df = pd.DataFrame(nouns, columns=['IMDB_ID', 'character_name', 'Nouns'])
chunks_df = pd.DataFrame(chunks, columns=['IMDB_ID', 'character_name', 'Chunks'])

In [ ]:
# Merge DataFrames on 'IMDB_ID' and 'character_name'
final_df = verbs_df.merge(adjs_df, on=['IMDB_ID', 'character_name']) \
                    .merge(nouns_df, on=['IMDB_ID', 'character_name']) \
                    .merge(chunks_df, on=['IMDB_ID', 'character_name'])

In [ ]:
# to have the gender 
final_df = pd.merge(final_df, output_characters_genders_context[['IMDB_ID', 'character_name', 'gender']], on=['IMDB_ID', 'character_name'], how='left')

previous cell gives df with columns IMDB_ID	character_name	Verbs	Adjectives	Nouns	Chunks	gender

In [ ]:
to_save_df = pd.merge(characters_data, final_df, on=['IMDB_ID', 'character_name'], how='left')

In [ ]:
to_save_df.to_csv('DATA/characters_personas_data.csv', index=False)

### CA DEVRAIT MARCHER !!!